In [ ]:
!pip install pandas scikit-learn gradio joblib


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving employeeattrition.csv to employeeattrition.csv


In [ ]:
import pandas as pd

# Replace 'employeeattrition.csv' with the exact filename you uploaded
df = pd.read_csv('employeeattrition.csv')

# Check the first 5 rows
df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical columns (like OverTime)
le = LabelEncoder()
df['OverTime'] = le.fit_transform(df['OverTime'])  # Yes=1, No=0

# Select features & target
X = df[['Age','MonthlyIncome','YearsAtCompany','OverTime','JobSatisfaction',
        'DistanceFromHome','NumCompaniesWorked','EnvironmentSatisfaction']]
y = df['Attrition'].map({'Yes':1,'No':0})

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Test accuracy
y_pred = lr.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8639455782312925


In [ ]:
import joblib

# Save the trained model
joblib.dump(lr, 'employee_attrition_model.joblib')

# Save the scaler
joblib.dump(scaler, 'scaler.joblib')

print("Model and Scaler saved successfully!")


Model and Scaler saved successfully!


In [ ]:
from google.colab import files

files.download('employee_attrition_model.joblib')
files.download('scaler.joblib')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import joblib
import gradio as gr

# Load the trained model and scaler
model = joblib.load('employee_attrition_model.joblib')
scaler = joblib.load('scaler.joblib')

# Prediction function
def predict_attrition(Age, MonthlyIncome, YearsAtCompany, OverTime, JobSatisfaction,
                      DistanceFromHome, NumCompaniesWorked, EnvironmentSatisfaction):
    # Create a dataframe from inputs
    input_df = pd.DataFrame([[Age, MonthlyIncome, YearsAtCompany, OverTime, JobSatisfaction,
                              DistanceFromHome, NumCompaniesWorked, EnvironmentSatisfaction]],
                            columns=['Age','MonthlyIncome','YearsAtCompany','OverTime','JobSatisfaction',
                                     'DistanceFromHome','NumCompaniesWorked','EnvironmentSatisfaction'])

    # Convert OverTime to numeric
    input_df['OverTime'] = 1 if OverTime=='Yes' else 0

    # Scale features
    input_scaled = scaler.transform(input_df)

    # Predict
    pred = model.predict(input_scaled)[0]

    return "⚠️ Employee is likely to RESIGN" if pred==1 else "✅ Employee is likely to STAY"

# Gradio interface
iface = gr.Interface(
    fn=predict_attrition,
    inputs=[
        gr.Number(label="Age"),
        gr.Number(label="MonthlyIncome"),
        gr.Number(label="YearsAtCompany"),
        gr.Dropdown(['Yes','No'], label="OverTime"),
        gr.Slider(1,4, step=1, label="JobSatisfaction"),
        gr.Number(label="DistanceFromHome"),
        gr.Number(label="NumCompaniesWorked"),
        gr.Slider(1,4, step=1, label="EnvironmentSatisfaction")
    ],
    outputs="text",
    title="Employee Attrition Prediction"
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://85330cb8f31442bc17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
